<a href="https://colab.research.google.com/github/f1th4p/kalkulator-python/blob/main/kalkulator_kwoty_na_fakturze.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# @title
# Automatyczna instalacja pakietów

%%capture
import sys
import subprocess

# Lista pakietów do zainstalowania
packages = ['scipy', 'numpy']

for package in packages:
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# Potwierdzenie, że pakiety zostały zainstalowane
print("Pakiety zostały zainstalowane.")

In [ ]:
from scipy.optimize import brentq
import ipywidgets as widgets
from IPython.display import display

def calculate_net_old(P, C, ZUS, target_net=6350.00, tax_rate=0.12, tax_deduction=300, health_rate=0.09):
    """
    Uproszczony model obliczeń (model z zeszłego roku):

      Przychód: P
      Koszty: C
      Społeczne: ZUS

      Dochód (podstawa) = P - C - ZUS
      Podatek = 0.12 * (P - C - ZUS) - 300
      Zdrowotna = 0.09 * (P - C - ZUS)
      Na rękę = (P - C - ZUS) - Podatek - Zdrowotna

    Funkcja zwraca różnicę między uzyskanym wynagrodzeniem netto a docelową kwotą (target_net).
    """
    base = P - C - ZUS
    tax = tax_rate * base - tax_deduction
    health = health_rate * base
    net = base - tax - health
    return net - target_net

# Parametry do wyboru:
target_net_default = 6350.00  # Domyślna kwota "na rękę"
C_example_default = 263.22    # Domyślne koszty
P_min, P_max = 9000, 10000  # Zakres poszukiwań przychodu

# Wybór ZUS
ZUS_options = {
    "ZUS bez chorobowego": 1646.47,
    "ZUS z chorobowym": 1773.96,
}

# Funkcja obliczająca przychód w zależności od wyboru ZUS
def calculate_income(ZUS_type, C_value, target_net_value):
    ZUS_value = ZUS_options[ZUS_type]
    P_solution = brentq(calculate_net_old, P_min, P_max, args=(C_value, ZUS_value, target_net_value))
    output_text = f"Rozwiązanie: Przychód = {P_solution:.2f} zł (ZUS: {ZUS_type}, Koszty: {C_value:.2f}, Kwota na rękę: {target_net_value:.2f})"
    print(output_text)

# Widget wyboru ZUS
ZUS_selector = widgets.Dropdown(
    options=ZUS_options.keys(),
    value="ZUS bez chorobowego",
    description='Wybierz ZUS:',
)

# Widget dla kosztów (z domyślną wartością)
C_selector = widgets.FloatText(
    value=C_example_default,
    description='Koszty:',
    step=1.0,
    min=0,
    max=5000
)

# Widget dla kwoty na rękę (z domyślną wartością)
target_net_selector = widgets.FloatText(
    value=target_net_default,
    description='Kwota na rękę:',
    step=10.0,
    min=0,
    max=20000
)

# Funkcja wywoływana przy każdej zmianie wartości w widżetach
def on_change(change):
    # Zaktualizowanie obliczeń przy każdej zmianie wartości
    calculate_income(ZUS_selector.value, C_selector.value, target_net_selector.value)

# Dodanie obserwatora dla każdego widżetu
ZUS_selector.observe(on_change, names='value')
C_selector.observe(on_change, names='value')
target_net_selector.observe(on_change, names='value')

# Wyświetlanie widżetów
display(ZUS_selector, C_selector, target_net_selector)